In [2]:
# Example in Chapter definition 4.10
import math
def calculate_uncertainty(probabilities, k):
    uncertainty = k * sum(p * math.log2(p) for p in probabilities)
    return uncertainty

# Example usage
probabilities = [0.7, 0.27, 0.03]
k = 1000
uncertainty = calculate_uncertainty(probabilities, k)
print("Uncertainty:", uncertainty)

# surprise: individual uncertainty for event 3
surprise = math.log2(probabilities[2])
print("Surprise:", surprise)




Uncertainty: -1021.989577307477
Surprise: -5.058893689053568


In [3]:
# Law 5.1

probabilities = [0.5, 0.2, 0.3]
k = 10000
uncertainty = calculate_uncertainty(probabilities, k)
print("Uncertainty:", uncertainty)

Uncertainty: -14854.752972273343
